In [12]:
import altair as alt
import altair_saver
import pandas as pd

In [21]:
alt.renderers.set_embed_options(scaleFactor=5)

RendererRegistry.enable('default')

In [22]:
df_caiso = pd.read_csv('/Users/meas/github/caiso-renewables/data/caiso_max_renewables_watch_2020-01-01_2020-12-31.csv')

In [23]:
step = 15
overlap = 1

chart = alt.Chart(df_caiso, height=step).transform_timeunit(
    Month='month(date)'
).transform_joinaggregate(
    mean_solar='mean(solar_pv)', groupby=['Month']
).transform_bin(
    ['bin_max', 'bin_min'], 'solar_pv'
).transform_aggregate(
    value='count()', groupby=['Month', 'mean_solar', 'bin_min', 'bin_max']
).transform_impute(
    impute='value', groupby=['Month', 'mean_solar'], key='bin_min', value=0
).mark_area(
    interpolate='monotone',
    fillOpacity=0.8,
    stroke='lightgray',
    strokeWidth=0.5
).encode(
    alt.X('bin_min:Q', 
#           scale=alt.Scale(domain=(4000, 12000)),
          bin='binned', title='Maximum Daily Solar Generation (MW)'),
    alt.Y(
        'value:Q',
        scale=alt.Scale(range=[step, -step * overlap]),
        axis=None
    ),
    alt.Fill(
        'mean_solar:Q',
        legend=None,
        scale=alt.Scale(domain=[4000, 12000], scheme='yelloworangebrown')
    )
).facet(
    row=alt.Row(
        'Month:T',
        title=None,
        header=alt.Header(labelAngle=0, labelAlign='left', format='%B')
    )
).properties(
    title='CAISO Solar Generation',
    bounds='flush'
).configure_facet(
    spacing=0
).configure_view(
    stroke=None
).configure_title(
    anchor='end'
)

In [24]:
chart

alt.FacetChart(...)

In [26]:
altair_saver.save(chart, 'chart.png')

WARN row encoding should be discrete (ordinal / nominal / binned).
WARN Channel x2 is required for "binned" bin.
